In [1]:
import numpy as np
import pandas as pd

from collections import defaultdict
from datetime import datetime as dt

from src.minimum_variance_delta import MinimumVarianceDelta, MinMax
from src.black_scholes import CallPut
from src.data import clear_cache

In [2]:
start_date = dt(2023, 1, 1)
end_date = dt(2023, 12, 31)

mvd = MinimumVarianceDelta(
    min_delta=0.35, 
    max_delta=0.45, 
    min_dte=15, 
    max_dte=45, 
    call_put=CallPut.CALL, 
    start_date=start_date, 
    end_date=end_date
)

In [6]:
coef = mvd.fit(rolling_window_size=None)
df = mvd.results_df(coefficients=coef)
y = mvd.y
y_hat = df['Y_HAT']

2023-01-04   -4.051637
2023-01-04   -4.754317
2023-01-04   -2.563441
2023-01-04   -4.372663
2023-01-04   -2.538304
                ...   
2023-12-28   -2.551189
2023-12-28   -2.641692
2023-12-28   -2.854782
2023-12-28   -3.393196
2023-12-28   -3.157528
Length: 15354, dtype: float64

In [5]:
mvd.results_df()

TypeError: results_df() missing 1 required positional argument: 'coefficients'

In [ ]:
start_date = dt(2023, 1, 1)
end_date = dt(2023, 12, 31)

# DTE buckets

# 15  -  45 days => 1 Month
# 65  - 105 days => 3 Months
# 165 - 195 days => 6 Months
# 240 - 300 days => 9 Months
# 335 - 395 days => 12 Months

# Delta buckets



call_delta_structure = {
    '10': MinMax(0.05, 0.15),
    '20': MinMax(0.15, 0.25),
    '30': MinMax(0.25, 0.35),
    '40': MinMax(0.35, 0.45),
    '50': MinMax(0.45, 0.55),
    '60': MinMax(0.55, 0.65),
    '70': MinMax(0.65, 0.75),
    '80': MinMax(0.75, 0.85),
    '90': MinMax(0.85, 0.95),
}

put_delta_structure = {
    '-10': MinMax(-0.15, -0.05),
    '-20': MinMax(-0.25, -0.15),
    '-30': MinMax(-0.35, -0.25),
    '-40': MinMax(-0.45, -0.35),
    '-50': MinMax(-0.55, -0.45),
    '-60': MinMax(-0.65, -0.55),
    '-70': MinMax(-0.75, -0.65),
    '-80': MinMax(-0.85, -0.75),
    '-90': MinMax(-0.95, -0.85),
}

In [ ]:
results = defaultdict(lambda: defaultdict(dict))
for term, (min_dte, max_dte) in term_structure.items():
    for delta, (min_delta, max_delta) in call_delta_structure.items():
        mvd = MVD(
            min_delta=min_delta, 
            max_delta=max_delta, 
            min_dte=min_dte, 
            max_dte=max_dte, 
            call_put=CallPut.CALL, 
            start_date=start_date, 
            end_date=end_date
        )
        results[term][delta] = mvd
    
    # For small machines (such as my laptop), it is recommended to clear the cache. We load call deltas first, then put deltas.
    # clear_cache()
    for delta, (min_delta, max_delta) in put_delta_structure.items():
        mvd = MVD(
            min_delta=min_delta, 
            max_delta=max_delta, 
            min_dte=min_dte, 
            max_dte=max_dte, 
            call_put=CallPut.PUT, 
            start_date=start_date, 
            end_date=end_date
        )
        results[term][delta] = mvd

In [ ]:
results['1M']['50'].df()[['EIV', 'DELTA']].plot.scatter(x='DELTA', y='EIV')

In [ ]:
# lets look at 1M
_data = results['1M']
df = pd.concat([_data[k].df(['EIV', 'DELTA']) for k in _data.keys()])
df.plot.scatter(x='DELTA', y='EIV')

In [ ]:
def _eiv(mvd):
    return mvd.df()['EIV'].mean()

x = sorted(call_delta_structure.keys() + put_delta_structure.keys())

fig, axes = plt.subplots(len(term_structure), len(x), figsize=(12, 12))

for i, ax in enumerate(axes.flat):
    term = x[i]
    y_values = [_eiv(results[term][k]) for k in x]
    ax.plot(x, y_values)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(f'Plot of x against y for term {term}')

plt.tight_layout()
plt.show()

x = sorted(list(results['1M'].keys()))
y = [_eiv(results['1M'][k]) for k in x]

import matplotlib.pyplot as plt

plt.plot(x, y)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Plot of x against y')
plt.show()

In [ ]:
deltas = np.arange(-1, 1, 0.1)


In [ ]:
deltas